In [ ]:
!pip install transformers
!pip install peft
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftConfig, PeftModel
import torch
import bitsandbytes
import accelerate
import json
from google.colab import drive
import librosa
from transformers import WhisperForConditionalGeneration, AutoProcessor
import os
from transformers import WhisperForConditionalGeneration, AutoProcessor
import torch
import librosa
from tqdm import tqdm
from pathlib import Path
import json
import numpy as np
import wave


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 11.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the model ID on the Hugging Face Hub
peft_model_id = "mirella-guenther/openai-whisper-large-v3-torgo-2-epochs"

# Load the PEFT configuration from the Hugging Face Hub
peft_config = PeftConfig.from_pretrained(peft_model_id)
processor = WhisperProcessor.from_pretrained(peft_config.base_model_name_or_path)

# Determine the device and data type
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16
# if device == "cuda" else torch.float32

# Load the Whisper model for conditional generation
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path,
    # load_in_8bit=True,
    device_map="auto",
    torch_dtype=dtype
)

# Apply the PEFT model
# model = PeftModel.from_pretrained(model, peft_model_id)
model.to(device)
print("Whisper model loaded successfully.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/771 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

Whisper model loaded successfully.


In [ ]:
file_path = '/content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/0015.wav'

In [ ]:
# normal_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3", torch_dtype=torch.bfloat16).to(device)
# normal_processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")
# audio, sr = librosa.load(file_path, sr=None)
# input = normal_processor(audio, return_tensors="pt", return_attention_mask=True, sampling_rate=16000)
# input = input.to(device)  # Move the input tensors to the same device as the model
# input = input.to(dtype=torch.bfloat16)  # Ensure the input tensors are of type BFloat16
# normal_output = normal_model.model.encoder(**input, output_hidden_states=True)
# output_2 = normal_model.model.encoder(**input, output_hidden_states=True)
# input = {k: v.to(device, dtype=torch.float16) for k, v in input.items()}
# output = model.model.encoder(**input, output_hidden_states=True)
# print(normal_output.last_hidden_state[0])
# print(output_2.last_hidden_state[0])
# print(output.last_hidden_state[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tensor([[-0.3457,  0.9297,  0.1709,  ..., -0.3789, -1.0000,  0.6250],
        [-0.1406, -0.7188, -0.2148,  ..., -0.2520,  0.6328, -0.2275],
        [-0.0109, -0.2012, -0.0337,  ...,  0.0737,  0.2734, -0.2363],
        ...,
        [-0.5195, -0.5508,  0.3340,  ..., -0.5156, -0.2021,  0.1167],
        [-0.3594, -0.5938, -0.1963,  ..., -0.0942,  0.2070, -0.6836],
        [-0.1074, -0.0771, -0.2480,  ..., -0.1943,  0.3574,  0.1270]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SelectBackward0>)
tensor([[-0.3457,  0.9297,  0.1709,  ..., -0.3789, -1.0000,  0.6250],
        [-0.1406, -0.7188, -0.2148,  ..., -0.2520,  0.6328, -0.2275],
        [-0.0109, -0.2012, -0.0337,  ...,  0.0737,  0.2734, -0.2363],
        ...,
        [-0.5195, -0.5508,  0.3340,  ..., -0.5156, -0.2021,  0.1167],
        [-0.3594, -0.5938, -0.1963,  ..., -0.0942,  0.2070, -0.6836],
        [-0.1074, -0.0771, -0.2480,  ..., -0.1943,  0.3574,  0.1270]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<Sele

In [ ]:
current_dir = os.getcwd()
# Navigate to the parent directory
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Define the path to the data folder
# data_folder = os.path.join(parent_dir, "data")

data_folder = '/content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/TORGO'

def get_wav_duration(file_path):
    with wave.open(file_path, 'r') as wav_file:
        frames = wav_file.getnframes()
        rate = wav_file.getframerate()
        duration = frames / float(rate)
    return duration

def find_wav_files(directory):
    wav_files = []
    # Iterate over all items (files and directories) in the current directory
    for item in os.listdir(directory):
        # Construct the full path
        item_path = os.path.join(directory, item)
        # If it's a file and ends with .wav, and there's no 'C' in the path, and duration is longer than 2 seconds, add it to the list
        if os.path.isfile(item_path) and item.lower().endswith(".wav"):
            if get_wav_duration(item_path) > 2.5:
                file = item_path
                if ('MC01' in file or 'FCO1' in file or 'MC04/Session1' in file or 'F04' in file or
                'M03' in file or 'F01' in file or 'M05' in file or 'F03' in file or
                 'M01' in file or 'M02' in file or 'M04' in file):
                    wav_files.append(item_path)
        # If it's a directory, recursively call the function
        elif os.path.isdir(item_path):
            wav_files.extend(find_wav_files(item_path))
    return wav_files

wav_files = find_wav_files(data_folder)
print(len(wav_files))

3672


In [ ]:
class_names = []
for i, file in enumerate(wav_files):
    # if ('MC' in file or 'FC01' in file):
        #  or 'FC' in file or 'Healthy' in file):
    if ('MC01' in file or 'FCO1' in file or 'MC04/Session1' in file):
        class_names.append('Healthy')
    elif ('F04' in file or 'M03' in file):
        class_names.append('VeryLow')
    elif ('M05' in file or 'F03' in file ):
        class_names.append('Low')
    elif ('F01' in file or 'M01' in file
          or 'M02' in file or 'M04' in file):
        class_names.append('Medium')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# directory = Path.cwd().parent.as_posix() + '/data/small_dataset'

audios = []
file_names = []
print('reading audio')
output_file_name = "/content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/TORGO_Embeddings_Mirella_Cut_New.json"
output_folder = "/content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/TORGO_Cut_Mirella_Embeddings_New"
with open(output_file_name, 'w') as json_file:
    json_file.write("[")  # Start of the JSON array
    # Iterate over files in the directory
    file_counter = 0
    for i, file_path in enumerate(wav_files):
        if file_path.endswith(".wav"):
            audio, sr = librosa.load(file_path, sr=None)
            # audios.append(audio)
            input = processor(audio, return_tensors="pt", return_attention_mask=True, sampling_rate=16000)
            input = {k: v.to(device, dtype=torch.float16) for k, v in input.items()}
            file_names.append(file_path)
            output = model.model.encoder(**input, output_hidden_states=True)
            # model.model.encoder(**input.to(dtype=torch.bfloat16), output_hidden_states=True)
            your_tensor = output.last_hidden_state[0].to(dtype=torch.float16)

            # Convert the tensor to a supported data type (e.g., float32) before converting to NumPy array
            your_tensor = your_tensor.float()  # Convert to float32
            # numpy_array = your_tensor.detach().numpy()
            numpy_array = your_tensor.detach().cpu().numpy()

            # ONLY TAKE FIRST 375 ie. first 7.5 seconds
            numpy_array = numpy_array[:125, :]

            if file_counter > 0:
               json_file.write(", ")  # Add comma before each element except the first one
            json.dump({"fileName": file_path, "numpyArray": numpy_array.tolist()}, json_file)

            file_name = str(i) +  ".json"
            entry_json_file_path = os.path.join(output_folder, file_name)
            with open(entry_json_file_path, 'x') as json_entry_file:
                json.dump({"numpyArray": numpy_array.tolist(), "className": class_names[i]}, json_entry_file)

            print('Written to json for file ' + file_path + ' which is file number ' +  str(file_counter))
            file_counter += 1
    json_file.write("]")  # End of the JSON array

cuda
reading audio
Written to json for file /content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/TORGO/F/F01/Session1/wav_arrayMic/0020.wav which is file number 0
Written to json for file /content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/TORGO/F/F01/Session1/wav_arrayMic/0127.wav which is file number 1
Written to json for file /content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/TORGO/F/F01/Session1/wav_arrayMic/0078.wav which is file number 2
Written to json for file /content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/TORGO/F/F01/Session1/wav_arrayMic/0077.wav which is file number 3
Written to json for file /content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/TORGO/F/F01/Session1/wav_arrayMic/0104.wav which is file number 4
Written to json for file /content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/TORGO/F/F01/Session1/wav_arrayMic/0031.wav which is file number 5
Written to json for file /content/drive/My Drive/CSE/Year 3/ResearchProject/Datas

In [ ]:
import os
print(os.listdir('/content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/'))

['0015.wav', 'Embeddings.json', 'train_data.list', 'TORGO', 'MSDM', 'TORGO_Embeddings_Mirella.json', 'random_data2.json', 'Mirella_Runs', '.ipynb_checkpoints', 'TORGO_Cut_Mirella_Embeddings', 'TORGO_Embeddings_Mirella_Cut.json']


In [ ]:
# Specify the file path
file_path = '/content/drive/My Drive/CSE/Year 3/ResearchProject/Dataset/TORGO_Embeddings_Mirella_Cut.json'

# Get the size of the file in bytes
file_size_bytes = os.path.getsize(file_path)

# Convert the size to gigabytes
file_size_gb = file_size_bytes / (1024 ** 3)

# Print the file size in gigabytes
print(f"The size of the file is: {file_size_gb:.6f} GB")

The size of the file is: 8.345087 GB
